In [6]:
!pip install music21

In [7]:
!unzip -q lakh-midi-clean.zip -d lakh_midi

In [8]:
!rm -rf midi_songs
!mkdir midi_songs


In [13]:
!find lakh_midi -name "*.mid" | head -n 300 | xargs -I {} cp -n {} midi_songs/


In [14]:
!ls midi_songs | wc -l


299


In [26]:
import os
import glob
import pickle
import numpy as np
from music21 import converter,instrument,note,chord,stream
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout,LSTM
from tensorflow.keras.utils import to_categorical


In [16]:
midi_files=glob.glob("midi_songs/*.mid")
print("Total MIDI files:",len(midi_files))


Total MIDI files: 299


In [17]:
notes=[]
for file in midi_files:
  midi=converter.parse(file)
  parts=instrument.partitionByInstrument(midi)
  if parts:
    elements=parts.parts[0].recurse()
  else:
    elements=midi.flat.notes
  for element in elements:
    if isinstance(element, note.Note):
      notes.append(str(element.pitch))
    elif isinstance(element, chord.Chord):
      notes.append('.'.join(str(n) for n in element.normalOrder))
print("Total notes:",len(notes))


/usr/local/lib/python3.12/dist-packages/music21/midi/translate.py:922: TranslateWarning: Unable to determine instrument from <music21.midi.MidiEvent SEQUENCE_TRACK_NAME, track=3, data=b'M\xe9lodie'>; getting generic Instrument
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/music21/midi/translate.py:922: TranslateWarning: Unable to determine instrument from <music21.midi.MidiEvent INSTRUMENT_NAME, track=3, data=b'M\xe9lodie'>; getting generic Instrument
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/music21/midi/translate.py:922: TranslateWarning: Unable to determine instrument from <music21.midi.MidiEvent SEQUENCE_TRACK_NAME, track=3, data=b'm\x82lodie'>; getting generic Instrument
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/music21/midi/translate.py:922: TranslateWarning: Unable to determine instrument from <music21.midi.MidiEvent SEQUENCE_TRACK_NAME, track=4, data=b'Guitare\xa0'>; getting generic Instrument
  warnings.warn(
/usr/local/lib/python3.12/d

Total notes: 81303


In [19]:
sequence_length=100
pitchnames=sorted(set(notes))
n_vocab=len(pitchnames)
note_to_int={note: number for number, note in enumerate(pitchnames)}
network_input=[]
network_output=[]
for i in range(len(notes)-sequence_length):
  seq_in=notes[i:i+sequence_length]
  seq_out=notes[i +sequence_length]
  network_input.append([note_to_int[n] for n in seq_in])
  network_output.append(note_to_int[seq_out])
n_patterns=len(network_input)

network_input=np.reshape(network_input,(n_patterns,sequence_length,1))
network_input=network_input/float(n_vocab)

network_output=to_categorical(network_output)

In [20]:
model=Sequential()
model.add(LSTM(256,input_shape=(network_input.shape[1], network_input.shape[2]),return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(256))
model.add(Dense(256,activation='relu'))
model.add(Dense(n_vocab,activation='softmax'))
model.compile(loss='categorical_crossentropy',optimizer='adam')
model.summary()

/usr/local/lib/python3.12/dist-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 100, 256)       │       264,192 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 100, 256)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 256)            │       525,312 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │        65,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 322)            │        82,754 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 938,050 (3.58 MB)

 Trainable params: 938,050 (3.58 MB)

 Non-trainable params: 0 (0.00 B)

In [21]:
model.fit(network_input,network_output,epochs=20,batch_size=64)

Epoch 1/20
1269/1269 ━━━━━━━━━━━━━━━━━━━━ 1517s 1s/step - loss: 4.3007
Epoch 2/20
1269/1269 ━━━━━━━━━━━━━━━━━━━━ 1485s 1s/step - loss: 3.9039
Epoch 3/20
1269/1269 ━━━━━━━━━━━━━━━━━━━━ 1521s 1s/step - loss: 3.7509
Epoch 4/20
1269/1269 ━━━━━━━━━━━━━━━━━━━━ 1570s 1s/step - loss: 3.5596
Epoch 5/20
1269/1269 ━━━━━━━━━━━━━━━━━━━━ 1549s 1s/step - loss: 3.3420
Epoch 6/20
1269/1269 ━━━━━━━━━━━━━━━━━━━━ 1569s 1s/step - loss: 3.0736
Epoch 7/20
1269/1269 ━━━━━━━━━━━━━━━━━━━━ 1585s 1s/step - loss: 2.7947
Epoch 8/20
1269/1269 ━━━━━━━━━━━━━━━━━━━━ 1556s 1s/step - loss: 2.5527
Epoch 9/20
1269/1269 ━━━━━━━━━━━━━━━━━━━━ 1573s 1s/step - loss: 2.3282
Epoch 10/20
1269/1269 ━━━━━━━━━━━━━━━━━━━━ 1584s 1s/step - loss: 2.1483
Epoch 11/20
1269/1269 ━━━━━━━━━━━━━━━━━━━━ 1582s 1s/step - loss: 1.9949
Epoch 12/20
1269/1269 ━━━━━━━━━━━━━━━━━━━━ 1623s 1s/step - loss: 1.8431
Epoch 13/20
1269/1269 ━━━━━━━━━━━━━━━━━━━━ 1569s 1s/step - loss: 1.7097
Epoch 14/20
1269/1269 ━━━━━━━━━━━━━━━━━━━━ 1552s 1s/step - loss: 1.5867
E

In [31]:
start=np.random.randint(0,len(network_input)-1)
pattern=network_input[start]
int_to_note={number:note for number,note in enumerate(pitchnames)}
prediction_output=[]
for i in range(200):
  prediction_input=np.reshape(pattern,(1,len(pattern),1))
  prediction_input=prediction_input/float(n_vocab)

  prediction=model.predict(prediction_input,verbose=0)
  index=np.argmax(prediction)
  result=int_to_note[index]
  prediction_output.append(result)

  pattern=np.append(pattern,index)
  pattern=pattern[1:]

In [32]:
offset=0
output_notes=[]
for pattern in prediction_output:
  if '.' in pattern or pattern.isdigit():
    notes_in_chord=pattern.split('.')
    chord_notes=[]
    for current_note in notes_in_chord:
      new_note=note.Note(int(current_note))
      new_note.storedInstrument=instrument.Piano()
      chord_notes.append(new_note)
    new_chord=chord.Chord(chord_notes)
    new_chord.offset=offset
    output_notes.append(new_chord)
  else:
    new_note=note.Note(pattern)
    new_note.offset=offset
    new_note.storedInstrument=instrument.Piano()
    output_notes.append(new_note)
  offset+=0.5
midi_stream=stream.Stream(output_notes)
midi_stream.write('midi',fp='generated_music.mid')



'generated_music.mid'

In [30]:
model.save("music_model.keras")